# Week 12 Problem Set

## Homework

**HW1.** *Comments:* Write a state machine whose inputs are the characters of a string. The string contains the code for a computer program. The output of the state machine are either:
- the input character if it is part of a comment, or
- `None`, otherwise.

Comment starts with a `#` character and continue to the end of the current line. If you want to create a string that contains a new line character, you can use `\n`.

For example,
```
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
print(m.transduce(inpstr))
```

The expected output is:
```
[None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]
```

You should start by drawing a state transition diagram indicating the states and what inputs cause transition to which other states. Use the test case above to determine if your state transition diagram is correct. You should begin writing your program only when you are confident that your diagram is correct. 

In [72]:
# Copy over the implementation of StateMachine from Cohort
from abc import ABC, abstractmethod

class StateMachine(ABC):
    def start(self):
        self.state = self.start_state
        pass

    def step(self, inp):
        self.state, output = self.get_next_values(self.state, inp)
        return output
        pass
        
    def transduce(self, inp_list):
        self.start()
        out_list = []
        for inp in inp_list:
            out_list.append(self.step(inp))
            if self.is_done():
                break
        return out_list
        pass

    @abstractmethod
    def get_next_values(self, state, inp):
        pass

    def done(self, state):
        return False

    def is_done(self):
        return self.done(self.state)

In [75]:
class CommentsSM(StateMachine):
    
    def __init__(self):
        self.start_state = ""
        pass

    def get_next_values(self, state, inp):
        if state == "#" or inp == "#":
            if inp == "\n":
                output = None
                next_state = ""
            else:
                output = inp
                next_state = "#"    
        else:
            output = None
            next_state = ""        
        return next_state, output

In [76]:
inpstr = "def func(x): # comment\n    return 1"
m = CommentsSM()
out = m.transduce(inpstr)
assert out == [None, None, None, None, None, None, None, None, None, None, None, None, None, "#", " ", "c", "o", "m", "m", "e", "n", "t", None, None, None, None, None, None, None, None, None, None, None, None, None]

In [59]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW2.** *First Word:* Write a state machine whose inputs are the characters of a string and which outputs either:
- the input character if it is part of the first word on a line, or
- `None`, otherwise

For the purposes here, a word is any sequence of consecutive characters that does not contain spaces or end-of-line characters. In this problem, comments have no special status. This means that if the line begins with `# `, then the first word is `#`. 

For example, 
```
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
print(m.transduce( inpstr))
```

The expected output is:
```
["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]
```

In [60]:
class FirstWordSM(StateMachine):
  
    def __init__(self):
        self.start_state = "\n"
        pass
  
    def get_next_values(self, state, inp):
        if state == "\n" or inp == "\n":
            if inp == " " or inp == "\n":
                output = None
                next_state = "\n"
            elif inp != " ":
                output = inp
                next_state = "w"
        elif state == "w":
            if inp == " ":
                output = None
                next_state = ""
            else:
                output = inp
                next_state = "w"
        else:
            output = None
            next_state = ""
        return next_state, output

In [61]:
inpstr = "def func(x): # comment\n    return 1"
m = FirstWordSM()
out = m.transduce(inpstr)
assert out == ["d", "e", "f", None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, "r", "e", "t", "u", "r", "n", None, None]


In [62]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW3.** *Robot:* Write a State Machine class that represent a robot. The dimension of the world and the robot initial position should be specified during the class instantiation. The robot can take in the following input:
- "left"
- "right"
- "up"
- "down"

The initial position of the robot is specified during the object instantiation and the input should modify the position of the robot. The robot position must not change if it exceed the boundary. At each step, the robot should output the updated position. 

In [63]:
class Position:
  
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
  
    def __str__(self):
        return f"({self.x:}, {self.y:})"

class Dimension:

    def __init__(self, width=0, height=0):
        self.width = width
        self.height = height
  
    def __str__(self):
        return f"width: {self.width:}, height: {self.height:}"

In [64]:
class RobotSM(StateMachine):

    def __init__(self, init_pos, dimension):
        self.world_dim = dimension
        self.start_state = init_pos
  
    def get_next_values(self, state, inp):        
        if inp == "right":
            state.x += 1
        elif inp == "left":
            state.x -= 1
        elif inp == "up":
            state.y += 1
        elif inp == "down":
            state.y -= 1
            
        if state.x < 0:
            state.x = 0
            
        if state.y < 0:
            state.y = 0
            
        if state.x > self.world_dim.width:
            state.x = self.world_dim.width
            
        if state.y > self.world_dim.height:
            state.y = self.world_dim.height
            
        return state, state

In [65]:
robot = RobotSM(Position(0, 0), Dimension(5, 5))
robot.start()
robot.transduce(["right", "right", "up", "up", "up", "left", "down"])
pos = robot.state
assert pos.x == 1 and pos.y == 2

In [66]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


**HW4.** *Search SM:* Write a function `sm_search` that takes in the following arguments:
- `sm_to_search`: is the State Machine instance to search. This argument is of the type `MapSM` as defined in CS4. You should use the `get_next_values()` of this State Machine instance to explore the next state in your search.
- `initial_state`: is the start state of the search. If it is not provided, it should be assigned to the `start_state` of sm_search. 
- `goal_test`: is a function that returns `True` if the argument is the end state of the search. If it is not provided, it should be eassigned to the `done` function of the state machine.

This function performs a **breadth-first-search** algorithm to explore the next states. 

The output is a `list` of `Step` instances from the `init_state` to the end state which is determined by the `goal_test` function.

This problem requires you to complete the following:
- `Queue` class from Problem Set 4 HW2.
- `MapSM` class in CS4.
- `SearchNode` and `Step` classes in CS5.

In [67]:
# Copy over the implementations of Queue from PS4 HW2
class Queue:
    def __init__(self):
        self.__items = []
    
    def enqueue(self, item):
        self.__items.append(item)
        pass
    
    def dequeue(self):
        if self.__items == []:
            return None
        else:
            return self.__items.pop(0)
        pass
    
    def peek(self):
        if self.is_empty == True:
            return None
        else:
            return self.__items[0]
        pass
    
    @property
    def is_empty(self):
        if self.__items == []:
            return True
        else:
            return False
        pass
    
    @property
    def size(self):
        return len(self.__items)
        pass

In [68]:
# Copy over the implementation of StateSpaceSearch from Cohort
from abc import abstractmethod

class StateSpaceSearch(StateMachine):
    @property
    @abstractmethod
    def statemap(self):
        pass

    @property
    @abstractmethod
    def legal_inputs(self):
        pass


In [77]:
# Copy over the implementation of MapSM from Cohort
class MapSM(StateSpaceSearch):
        
    def __init__(self, start):
        self.start_state = start

    @property
    def statemap(self):
        statemap = {"S": ["A", "B"],
                    "A": ["S", "C", "D"],
                    "B": ["S", "D", "E"],
                    "C": ["A", "F"],
                    "D": ["A", "B", "F", "H"],
                    "E": ["B", "H"],
                    "F": ["C", "D", "G"],
                    "H": ["D", "E", "G"],
                    "G": ["F", "H"]}
        return statemap

    @property
    def legal_inputs(self):
        state_length = [len(state) for state in self.statemap.values()]
        return set(range(max(state_length)))
        pass

  
    def get_next_values(self, state, inp):
        if state in self.statemap:
            legal_list = self.statemap[state]
            if inp < len(legal_list):
                next_state = legal_list[inp]
                output = next_state
                return next_state, output
        return state, False
        pass

In [78]:
# Copy over the implementations of Step and SearchNode from Cohort
class Step:
    def __init__(self, action, state):
        self.action = action
        self.state = state
    
    def __eq__(self, other):
        return self.action == other.action and self.state == other.state
  
    def __str__(self):
        return f"action: {self.action:}, state: {self.state:}"

class SearchNode:
    def __init__(self, action, state, parent):
        self.state = state
        self.action = action
        self.parent = parent
  
    def path(self):
        if self.parent == None:
            return [Step(None, self.state)]
        else:
            return self.parent.path() + [Step(self.action, self.state)]
        
    def in_path(self, state):
        if self.state == state:
            return True
        elif self.parent == None:
            return False
        else:
            return self.parent.in_path(state)
  
    def __eq__(self, other):
        if self is None and other is None:
            return True
        elif self is None:
            return False
        elif other is None:
            return False
        else:
            return self.state == other.state and self.parent == other.parent and \
                   self.action == other.action

In [113]:
def sm_search(sm_to_search, initial_state=None, goal_test=None):
    # check if initial_state is provided
    # if it is, use it
    # otherwise, get the start state of sm_to_search
    if initial_state == None:
        # replace None to take the start state of sm_to_search
        init_state = sm_to_search.start_state
    else:
        init_state = initial_state
  
    # check if goal_test is provided
    # if it is, use it
    # otherwise, use the done method as the goal function
    # taken from sm_to_search
    if goal_test == None:
        goal_func = sm_to_search.done
    else:
        goal_func = goal_test
      
    # create a Queue instance to store the node to explore
    # replace the None below
    agenda = Queue()
  
    # if the initial state is the goal state, 
    # then we are done and exit
    if goal_func(init_state):
        return [Step(None, init_state)]
  
    # otherwise, add the current node into the agenda 
    agenda.enqueue(SearchNode(None, init_state, None))
    
    # explore as long as the Queue is not empty
    while not agenda.is_empty:
        
        # replace None to take out the parent from the Queue
        parent = agenda.dequeue()
        
        # create a list to keep track which child state have been explored
        new_child_state = []
        
        # get all the legal input values
        actions = sm_to_search.legal_inputs
        
        #iterate over all legal inputs
        for a in actions:
            # get the next possible state using the current action
            # call get_next_values to get the next state
            # replace the None below
            new_s, output = sm_to_search.get_next_values(parent.state, a)
            
            # create a new search node from the new_s
            # replace the None below
            new_n = SearchNode(a, new_s, parent)
            
            # if the new state is the goal state, then we exit and return the path
            if goal_func(new_s):
                return new_n.path()
            
            # if the new state is already in the list of new child state, ignore it
            elif new_s in new_child_state:
                pass
            
            # if the new state is in the path of the current node, ignore it
            elif parent.in_path(new_s):
                pass
            
            # otherwise, add the new state into the list
            # and the new node into the Queue
            else:
                # step 1. add the new state into the new_child_state
                new_child_state.append(new_s)
                
                # step 2. add the new node into the Queue
                agenda.enqueue(new_n)
                pass
    return None

In [114]:
mapSM = MapSM("S")
ans = sm_search(mapSM , "S" , lambda s: s=="H" )
steps = [(step.action, step.state) for step in ans]
assert steps == [(None, "S"), (0, "A"), (2, "D"), (3, "H")]
for step in ans:
    print(step)

action: None, state: S
action: 0, state: A
action: 2, state: D
action: 3, state: H


In [115]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
